In [1]:
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt
from models.Unet_Resnet import Unet_Resnet50

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
model_dir = os.path.join(os.getcwd(), 'config')
unet_model = Unet_Resnet50(model_dir = model_dir,
                           visible_gpu = 0,
                           config_filepath=os.path.join(model_dir, 'Unet_Resnet50-20191017T0347-config.yml'),
                           for_prediction=True, 
                           tile_size = [512,512]
                          )
unet_model.initialize_model()
unet_model.load_weights(os.path.join(model_dir, 'Unet_Resnet50-20191017T0347_0155.h5'))


Loaded config file from /gpfs0/home/mbigwj/Documents/WJ_analysis/Actin_segmentation_pub/config/Unet_Resnet50-20191017T0347-config.yml
Updating model_dir to /gpfs0/home/mbigwj/Documents/WJ_analysis/Actin_segmentation_pub/config


Unet_Resnet50 2020-12-18 16:39:47.639897

Predicting using single GPU or CPU..
Using 1 - Dice + BCE loss
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loaded weights from: /gpfs0/home/mbigwj/Documents/WJ_analysis/Actin_segmentation_pub/config/Unet_Resnet50-20191017T0347_0155.h5


In [8]:
root = '/mnt/mbi/images/sesha/WJ/perkin_elmer_exports/01Dec2020-fixed cell-exported'
os.path.isdir(root)
os.listdir(root)


['._.DS_Store',
 '01dec2020_siVasp_LatA',
 '01dec2020_siMyo1c_LatA',
 '01dec2020_siMyo1d',
 '01dec2020_siMyo1d_LatA',
 '01dec2020_sidaam2-LatA',
 '01dec2020_Control-LatA',
 '01dec2020_siMyo1c',
 '.DS_Store',
 '01dec2020_Control',
 '01dec2020_siVasp',
 '01dec2020_siFMN2',
 '01dec2020_siFMN2_latA',
 '01dec2020_sidaam2']

In [9]:
#  Check incomplete segmentation
warnings.simplefilter(action='ignore', category=UserWarning)
exceptions = []

actin_folders = sorted([i for i, _, _ in os.walk(root) if i.endswith('actin')])
for folder in actin_folders:
    maskdir = os.path.join(folder, 'Masks/Unet_Resnet50-20191017T0347_0155')
    try:  
        print(folder)
        temp = unet_model.predict_images(folder)
    except Exception as e:
        exceptions.append((e, folder))

if exceptions:
    for exception, folder in exceptions:
        print(exception, folder)

print("Done")
       

/mnt/mbi/images/sesha/WJ/perkin_elmer_exports/01Dec2020-fixed cell-exported/01dec2020_Control-LatA/actin
1/1 [==============================] - 0s 36ms/step
/mnt/mbi/images/sesha/WJ/perkin_elmer_exports/01Dec2020-fixed cell-exported/01dec2020_Control/actin
1/1 [==============================] - 0s 39ms/step
/mnt/mbi/images/sesha/WJ/perkin_elmer_exports/01Dec2020-fixed cell-exported/01dec2020_siFMN2/actin
1/1 [==============================] - 0s 35ms/step
/mnt/mbi/images/sesha/WJ/perkin_elmer_exports/01Dec2020-fixed cell-exported/01dec2020_siFMN2_latA/actin
1/1 [==============================] - 0s 36ms/step
/mnt/mbi/images/sesha/WJ/perkin_elmer_exports/01Dec2020-fixed cell-exported/01dec2020_siMyo1c/actin
1/1 [==============================] - 0s 37ms/step
/mnt/mbi/images/sesha/WJ/perkin_elmer_exports/01Dec2020-fixed cell-exported/01dec2020_siMyo1c_LatA/actin
1/1 [==============================] - 0s 36ms/step
/mnt/mbi/images/sesha/WJ/perkin_elmer_exports/01Dec2020-fixed cell-exported

In [10]:
unet_model.end_training()